In [250]:
import math
import numpy as np
import matplotlib.pyplot as plt
import random
class QuantumCircuit:
    def __init__(self, n):
        self.n = n
        # self.state = {}
        self.initialize_state()
    def initialize_state(self, i=0):
        key = format(i, f'0{self.n}b')
        self.state = {key: 1.0}

    def x(self, i):
        new_state = {}
        for key, value in self.state.items():
            flipped_key = key[:self.n -i-1] + ('1' if key[self.n -i-1] == '0' else '0') + key[self.n -i:]
            new_state[flipped_key] = value
        self.state = new_state

    def z(self, i):
        
        for key in self.state:
            if key[self.n -i-1] == '1':
                self.state[key] *= -1

    def t(self, i):
      
        for key in self.state:
            if key[self.n -i-1] == '1':
                self.state[key] *= np.exp(1j * np.pi / 4)

    def tdag(self, i):
        
        for key in self.state:
            if key[self.n -i-1] == '1':
                self.state[key] *= np.exp(-1j * np.pi / 4)

    def cx(self, control, target):
        new_state = {}
        for key, value in self.state.items():
            if key[self.n -control -1] == '1':
                flipped_key = key[:self.n -target -1] + ('1' if key[self.n -target-1] == '0' else '0') + key[self.n -target:]
                new_state[flipped_key] = value  # Reverse the binary string before adding
            else:
                new_state[key] = value  # Reverse the binary string before adding
        self.state = new_state

    def h(self, i):
        new_state = {}
        for key, value in self.state.items():
            key_0 = key[:self.n -i-1]+'0' +key[self.n -i:]
            key_1 = key[:self.n -i-1]+'1' +key[self.n -i:]

            # Add or update the amplitude for key_0
            new_state[key_0] = new_state.get(key_0, 0) + value / np.sqrt(2)

            # Add or update the amplitude for key_1, with a sign flip if the original bit was 1
            new_state[key_1] = new_state.get(key_1, 0) + (-value if key[self.n -i -1] == '1' else value) / np.sqrt(2)
        
        self.state = new_state




    def measure(self, i, shots=1):
        counts = {'0': 0, '1': 0}
    
        for _ in range(shots):
            # Calculate probabilities
            probability_of_1 = 0.0
            for key, value in self.state.items():
                if key[self.n - 1 - i] == '1':
                    probability_of_1 += abs(value)**2
    
            # Randomly decide the measurement outcome
            measurement = '1' if random.random() < probability_of_1 else '0'
            counts[measurement] += 1
    
            if shots == 1:
                # Collapse and normalize the state for a single shot
                new_state = {}
                for key, value in self.state.items():
                    if key[self.n - 1 - i] == measurement:
                        new_state[key] = value
    
                norm_factor = math.sqrt(sum(abs(value)**2 for value in new_state.values()))
                for key in new_state:
                    new_state[key] /= norm_factor
    
                self.state = new_state
                return new_state 
        prob =  {key: value / shots for key, value in counts.items()}
        return counts, prob

    def cleanup_state(self, threshold=1e-6,  decimals=5):
        
        new_state = {}
        for key, value in self.state.items():
            rounded_value = round(value.real, decimals) + round(value.imag, decimals) * 1j
            if abs(rounded_value) >= threshold:
                new_state[key] = rounded_value
        
        self.state = new_state if new_state else self.initialize_state()




In [155]:
# Example usage
qc = QuantumCircuit(2)
qc.initialize_state(1)

qc.h(0)
qc.h(1)
qc.t(0)
qc.t(1)
print(qc.state)
# counts, prob = qc.measure(1, shots=100)  # Measure qubit 1 with 10 shots

# # Plotting the results
# plt.bar(prob.keys(), prob.values())
# plt.xlabel('State')
# plt.ylabel('Counts')
# plt.title('Measurement Results')
# plt.show()
# print(prob)

{'00': 0.49999999999999994, '10': (0.35355339059327373+0.35355339059327373j), '01': (-0.35355339059327373-0.35355339059327373j), '11': -0.5j}


In [270]:
def execute_qasm(file_path):
    # First, determine the number of qubits from the qreg declaration
    n_qubits = 0
    f = open(file_path, "r")
    file = f.readlines()
    for line in file:
        # Remove comments and whitespace
        line = line.split("//")[0].strip()

        # Skip empty lines and non-qreg declarations
        if not line or 'OPENQASM' in line or 'include' in line or 'creg' in line:
            continue

        if 'qreg' in line:
            # Format 'qreg q[n];'
            n_qubits = int(line.split('[')[1].split(']')[0])
            # print(n_qubits)
            break

    # Initialize the QuantumCircuit with the correct number of qubits
    qc = QuantumCircuit(n_qubits)
    # print(qc.state)
    # Now execute the operations
    with open(file_path, 'r') as file:
        for line in file:
            # Remove any comments and whitespace
            line = line.split("//")[0].strip()

            # Skip non-gate lines
            if not line or 'OPENQASM' in line or 'include' in line or 'qreg' in line or 'creg' in line:
                continue

            # Break the line into operation and its parameters
            parts = line.split()
            operation = parts[0]

            if operation == 'cx':
                control, target = map(lambda x: int(x.split('[')[1].split(']')[0]), parts[1].split(','))
                qc.cx(control, target)
            elif operation == 'x':
                target = int(parts[1].split('[')[1].split(']')[0])
                qc.x(target)
            elif operation == 'z':
                target = int(parts[1].split('[')[1].split(']')[0])
                qc.z(target)
            elif operation == 'h':
                target = int(parts[1].split('[')[1].split(']')[0])
                qc.h(target)
            elif operation == 't':
                target = int(parts[1].split('[')[1].split(']')[0])
                qc.t(target)
            elif operation == 'tdg':
                target = int(parts[1].split('[')[1].split(']')[0])
                qc.tdag(target)
            else:
                print(f"Unknown operation: {operation}")
                
            # print("State after operation:", qc.state)
    qc.cleanup_state()
    return print(qc.state)

# # Example usage
# qc = execute_qasm('miller_11.qasm')


In [271]:
# Example usage
qc = execute_qasm('miller_11.qasm')

{'0000000000000000': (1+0j)}


In [253]:
qc = execute_qasm('alu-bdd_288.qasm')

{'0000000001000000': (1+0j)}


In [254]:
qc = execute_qasm('cm152a_212.qasm')

{'0000000000001110': (1+0j)}


In [255]:
qc = execute_qasm('cnt3-5_179.qasm')

{'0000000000000000': (1+0j)}


In [256]:
qc = execute_qasm('con1_216.qasm')

{'0000000011101001': (1+0j)}


In [257]:
qc = execute_qasm('decod24-v2_43.qasm')

{'0000000000001000': (1+0j)}


In [258]:
qc = execute_qasm('f2_232.qasm')

{'0000000001100000': (1+0j)}


In [259]:
qc = execute_qasm('hwb5_53.qasm')

{'0000000000000000': (1+0j)}


In [260]:
qc = execute_qasm('mini_alu_305.qasm')

{'0000000110000000': (1+0j)}


In [261]:
qc = execute_qasm('one-two-three-v3_101.qasm')

{'0000000000000011': (1+0j)}


In [262]:
qc = execute_qasm('rd84_142.qasm')

{'0000000000000000': (1+0j)}


In [263]:
qc = execute_qasm('squar5_261.qasm')

{'0001101100000000': (1+0j)}


In [264]:
qc = execute_qasm('sym6_316.qasm')

{'0001000000000000': (1+0j)}


In [265]:
qc = execute_qasm('wim_266.qasm')

{'0000011111111011': (1+0j)}
